# 基于MindNLP的Roberta模型Prompt Tuning

该实验可进行在线体验，在线体验链接（https://pangu.huaweicloud.com/gallery/asset-detail.html?id=016991f8-0e0d-44c8-96f7-8b2cad54c592
）

## 环境安装

运行如下两个代码块，创建python-3.9.0 kernel。

> 注意：
> 
>此为在线运行平台配置python3.9的指南，如在其他环境平台运行案例，请根据实际情况修改如下代码
> 
> 以下两个代码块仅能运行一次，多次运行会出现kernel报错。
>
> 如出现多次运行导致的kernel报错，请终止实例（点击右上角“停止NoteBook实例”的圆形图标），并重启实例。

In [1]:
%%capture captured_output
!/home/ma-user/anaconda3/bin/conda create -n python-3.9.0 python=3.9.0 -y --override-channels --channel https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main
!/home/ma-user/anaconda3/envs/python-3.9.0/bin/pip install ipykernel

In [2]:
import json
import os

data = {
   "display_name": "python-3.9.0",
   "env": {
      "PATH": "/home/ma-user/anaconda3/envs/python-3.9.0/bin:/home/ma-user/anaconda3/envs/python-3.7.10/bin:/modelarts/authoring/notebook-conda/bin:/opt/conda/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/home/ma-user/modelarts/ma-cli/bin:/home/ma-user/modelarts/ma-cli/bin"
   },
   "language": "python",
   "argv": [
      "/home/ma-user/anaconda3/envs/python-3.9.0/bin/python",
      "-m",
      "ipykernel",
      "-f",
      "{connection_file}"
   ]
}

if not os.path.exists("/home/ma-user/anaconda3/share/jupyter/kernels/python-3.9.0/"):
    os.mkdir("/home/ma-user/anaconda3/share/jupyter/kernels/python-3.9.0/")

with open('/home/ma-user/anaconda3/share/jupyter/kernels/python-3.9.0/kernel.json', 'w') as f:
    json.dump(data, f, indent=4)

创建完成后，稍等片刻，或刷新页面。如下图所示，点击右上角（或左上角）kernel选择python-3.9.0。

![change-kernel](https://mindspore-demo.obs.cn-north-4.myhuaweicloud.com/imgs/ai-gallery/change-kernel.PNG)

安装mindspore, mindnlp及其他依赖

In [1]:
%%capture captured_output

!pip install https://ms-release.obs.cn-north-4.myhuaweicloud.com/2.2.14/MindSpore/unified/x86_64/mindspore-2.2.14-cp39-cp39-linux_x86_64.whl --trusted-host ms-release.obs.cn-north-4.myhuaweicloud.com -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install mindnlp
!pip install ipywidgets
!pip install tqdm==4.65.0
!pip install scikit-learn

In [2]:
%env HF_ENDPOINT=https://hf-mirror.com

env: HF_ENDPOINT=https://hf-mirror.com


## 模型与数据集加载

本案例对roberta-large模型基于GLUE基准数据集进行prompt tuning。

In [4]:
import argparse
import os

import mindspore
from mindspore.experimental.optim import AdamW
from tqdm import tqdm
import evaluate
from mindnlp.dataset import load_dataset
from mindnlp.engine import set_seed
from mindnlp.transformers import AutoModelForSequenceClassification, AutoTokenizer
from mindnlp.modules.optimization import get_linear_schedule_with_warmup
from mindnlp.peft import (
    get_peft_config,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    PeftType,
    PromptTuningConfig,
)

In [5]:
batch_size = 32
model_name_or_path = "AI-ModelScope/roberta-large"
task = "mrpc"
peft_type = PeftType.PROMPT_TUNING
# num_epochs = 20
num_epochs = 5

prompt tuning配置，任务类型选为"SEQ_CLS", 即序列分类。

In [6]:
# peft config
peft_config = PromptTuningConfig(task_type="SEQ_CLS", num_virtual_tokens=10)
# learning rate
lr = 1e-3

加载tokenizer。如模型为GPT、OPT或BLOOM类模型，从序列左侧添加padding，其他情况下从序列右侧添加padding。

In [7]:
# load tokenizer
if any(k in model_name_or_path for k in ("gpt", "opt", "bloom")):
    padding_side = "left"
else:
    padding_side = "right"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, padding_side=padding_side, mirror="modelscope")
if getattr(tokenizer, "pad_token_id") is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

  0%|          | 0.00/482 [00:00<?, ?B/s]

  0%|          | 0.00/878k [00:00<?, ?B/s]

  0%|          | 0.00/446k [00:00<?, ?B/s]

  0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [8]:
datasets = load_dataset("glue", task)
print(next(datasets['train'].create_dict_iterator()))

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

{'sentence1': Tensor(shape=[], dtype=String, value= 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .'), 'sentence2': Tensor(shape=[], dtype=String, value= 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .'), 'label': Tensor(shape=[], dtype=Int64, value= 1), 'idx': Tensor(shape=[], dtype=Int64, value= 0)}


In [9]:
from mindnlp.dataset import BaseMapFuction

class MapFunc(BaseMapFuction):
    def __call__(self, sentence1, sentence2, label, idx):
        outputs = tokenizer(sentence1, sentence2, truncation=True, max_length=None)
        return outputs['input_ids'], outputs['attention_mask'], label


def get_dataset(dataset, tokenizer):
    input_colums=['sentence1', 'sentence2', 'label', 'idx']
    output_columns=['input_ids', 'attention_mask', 'labels']
    dataset = dataset.map(MapFunc(input_colums, output_columns),
                          input_colums, output_columns)
    dataset = dataset.padded_batch(batch_size, pad_info={'input_ids': (None, tokenizer.pad_token_id),
                                                         'attention_mask': (None, 0)})
    return dataset

train_dataset = get_dataset(datasets['train'], tokenizer)
eval_dataset = get_dataset(datasets['validation'], tokenizer)

In [10]:
print(next(train_dataset.create_dict_iterator()))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': Tensor(shape=[32, 70], dtype=Int64, value=
[[    0, 10127,  1001 ...     1,     1,     1],
 [    0,   975, 26802 ...     1,     1,     1],
 [    0,  1213,    56 ...     1,     1,     1],
 ...
 [    0,   133,  1154 ...     1,     1,     1],
 [    0, 12667,  8423 ...     1,     1,     1],
 [    0, 32478,  1033 ...     1,     1,     1]]), 'attention_mask': Tensor(shape=[32, 70], dtype=Int64, value=
[[1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0],
 ...
 [1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0]]), 'labels': Tensor(shape=[32], dtype=Int64, value= [1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 
 1, 1, 0, 0, 1, 1, 1, 0])}


In [11]:
metric = evaluate.load("glue", task)

加载模型并打印微调参数量，可以看到仅有不到0.6%的参数参与了微调。

如出现如下告警请忽略，并不影响模型的微调。

```text
The following parameters in checkpoint files are not loaded:
['lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'roberta.embeddings.position_ids']
The following parameters in models are missing parameter:
['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
```

In [12]:
# load model
model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path, return_dict=True, mirror="modelscope")
model = get_peft_model(model, peft_config)
# print number of trainable parameters
model.print_trainable_parameters()

  0%|          | 0.00/1.32G [00:00<?, ?B/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The following parameters in checkpoint files are not loaded:
['lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'roberta.embeddings.position_ids']
The following parameters in models are missing parameter:
['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']


trainable params: 2,113,540 || all params: 356,423,684 || trainable%: 0.5929852854559463


## 模型微调（prompt tuning）

指定优化器和学习率调整策略

In [13]:
optimizer = AdamW(params=model.trainable_params(), lr=lr)

# Instantiate scheduler
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0.06 * (len(train_dataset) * num_epochs),
    num_training_steps=(len(train_dataset) * num_epochs),
)

打印参与微调的模型参数

In [14]:
# print name of trainable parameters
model.trainable_params()

[Parameter (Tensor(shape=[1024, 1024], dtype=Float32, value=[...], name=base_model.classifier.original_module.dense.weight), requires_grad=True),
 Parameter (Tensor(shape=[1024], dtype=Float32, value=[...], name=base_model.classifier.original_module.dense.bias), requires_grad=True),
 Parameter (Tensor(shape=[2, 1024], dtype=Float32, value=[...], name=base_model.classifier.original_module.out_proj.weight), requires_grad=True),
 Parameter (Tensor(shape=[2], dtype=Float32, value=[0. 0.], name=base_model.classifier.original_module.out_proj.bias), requires_grad=True),
 Parameter (Tensor(shape=[1024, 1024], dtype=Float32, value=[...], name=base_model.classifier.modules_to_save.default.dense.weight), requires_grad=True),
 Parameter (Tensor(shape=[1024], dtype=Float32, value=[...], name=base_model.classifier.modules_to_save.default.dense.bias), requires_grad=True),
 Parameter (Tensor(shape=[2, 1024], dtype=Float32, value=[...], name=base_model.classifier.modules_to_save.default.out_proj.weight

按照如下步骤定义训练逻辑：

1. 构建正向计算函数
2. 函数变换，获取微分函数
3. 定义训练一个step的逻辑
4. 遍历训练数据集进行模型训练，同时每一个epoch后，遍历验证数据集获取当前的评价指标（accuracy、f1 score）

In [15]:
# define forward function
def forward_fn(**batch):
    outputs = model(**batch)
    loss = outputs.loss
    return loss

# Get gradient function
grad_fn = mindspore.value_and_grad(forward_fn, None, model.trainable_params())

# Define function of one-step training
def train_step(**batch):
    loss, grads = grad_fn(**batch)
    optimizer(grads)
    return loss

# Start training
for epoch in range(num_epochs):
    model.set_train()
    train_total_size = train_dataset.get_dataset_size()
    # Iterate through the dataset
    for step, batch in enumerate(tqdm(train_dataset.create_dict_iterator(), total=train_total_size)):
        loss = train_step(**batch)
        lr_scheduler.step()

    # Evaluate while training
    model.set_train(False)
    eval_total_size = eval_dataset.get_dataset_size()
    for step, batch in enumerate(tqdm(eval_dataset.create_dict_iterator(), total=eval_total_size)):
        outputs = model(**batch)
        predictions = outputs.logits.argmax(axis=-1)
        predictions, references = predictions, batch["labels"]
        metric.add_batch(
            predictions=predictions,
            references=references,
        )
    
    # Calculate accuracy and f1 score
    eval_metric = metric.compute()
    print(f"epoch {epoch}:", eval_metric)

100%|██████████| 13/13 [00:05<00:00,  2.31it/s]


epoch 0: {'accuracy': 0.6911764705882353, 'f1': 0.8130563798219584}


100%|██████████| 13/13 [00:05<00:00,  2.56it/s]


epoch 1: {'accuracy': 0.6985294117647058, 'f1': 0.8150375939849624}


100%|██████████| 13/13 [00:05<00:00,  2.57it/s]


epoch 2: {'accuracy': 0.7083333333333334, 'f1': 0.8199697428139183}


100%|██████████| 13/13 [00:05<00:00,  2.54it/s]


epoch 3: {'accuracy': 0.7083333333333334, 'f1': 0.8221225710014948}


100%|██████████| 13/13 [00:05<00:00,  2.55it/s]

epoch 4: {'accuracy': 0.7058823529411765, 'f1': 0.8198198198198198}
